# Implantation de Groth16 en Python : Un guide pas à pas

Nous explorons l'implantation de Groth16 en Python. Il s'agit d'un protocole de preuve à divulgation nulle (zk-SNARK) largement utilisée en cryptographie. L'objectif est de rendre l'implantation facile à comprendre pour tous les développeurs. Le code est proposé sous la forme d'un notebook permettant d'exécuter chaque cellule indépendamment.

## Qu'est-ce que Groth16 ?

C'est un protocole de la famille zk-SNARK (Ref ?) qui permet de prouver la connaissance d'une solution à un problème sans révéler cette solution. Il est utilisé dans de nombreux domaines, tels que la confidentialité des données, la vérification déléguée et la blockchain. L'implantation de Groth16 en Python nous permet de mieux comprendre son fonctionnement et de l'adapter à nos propres besoins.

## Pourquoi utiliser Python ?

C'est un langage de programmation populaire et largement utilisé dans le domaine de la science des données et de l'apprentissage automatique. Sa syntaxe simple et expressive en fait un choix idéal pour implémenter des algorithmes complexes tels que Groth16. De plus, la richesse de la bibliothèque standard de Python et la disponibilité de nombreuses bibliothèques tierces en font un outil puissant pour la mise en œuvre de protocoles cryptographiques.

## Prérequis

Avant de commencer, assurez-vous d'avoir une connaissance de base de Python et des concepts de cryptographie. Nous utilisons certaines bibliothèques Python couramment utilisées en cryptographie, telles que `numpy`, `galois` ou `py_ecc`. Ici, nous n'allons pas expliquer pourquoi groth16 a été conçu de cette façon, mais plutôt comment il fonctionne. Si vous souhaitez en savoir plus sur les détails de conception, consultez le papier scientifique [Groth16](https://eprint.iacr.org/2016/260.pdf) de sont autheur Jens Groth.

## Structure de l'article

Dans cet article, nous allons suivre une approche pas à pas pour implémenter Groth16 en Python. Nous commençons par importer les bibliothèques nécessaires et définir les paramètres du protocole. Ensuite, nous générerons l'ensemble des données utiles pour les différents composants qui s'échangent de l'information : le trusted setup, le prouver et le verifier. Lorsque nous nous sommes penchés sur ce projet d'implémenté Groth16 en python, nous avons vu qu'il existait quelques ressources sur le web, mais qui ne respectait pas toujours les critères du protocole.

## Quelques expliquations

Cette implantation est réalisée dans le cadre d'un projet recherche sur les protocoles de preuve à divulgation nulle (zk-SNARK) et leurs applications à la blockchain. Dans un premier temps, nous avons fait un état de l'art des ZKP et nous nous sommes penchés sur groth16. Notre implantation a été réalisée à partir de plusieurs ressources, telles que :
- Le papier de Jens Groth (https://medium.com/r/?url=https%3A%2F%2Feprint.iacr.org%2F2016%2F260.pdf)
- Les articles de blog de Vitalik: part1(https://medium.com/@VitalikButerin/quadratic-arithmetic-programs-from-zero-to-hero-f6d558cea649), part2(https://medium.com/@VitalikButerin/exploring-elliptic-curve-pairings-c73c1864e627), part3(https://medium.com/@VitalikButerin/zk-snarks-under-the-hood-b33151a013f6). 
- zkSNARKs: R1CS and QAP(https://medium.com/r/?url=https%3A%2F%2Frisencrypto.github.io%2FzkSnarks%2F)
- Under the hood of zkSNARK Groth16 protocol (part 1)(https://medium.com/coinmonks/under-the-hood-of-zksnark-groth16-protocol-part-5-2a958f7051c2)
- Zkbook de rareskills(https://medium.com/r/?url=https%3A%2F%2Fwww.rareskills.io%2Fpost%2Fgroth16)

# Plan general de l'implantation:

L'objectif du protocole est de pouvoir générer une preuve de connaissance sans divulgation d'une solution à un problème NP. Dans notre cas, nous souhaitons prouver que nous connaissons un polynôme quelconque. Pour cela, nous allons suivre les étapes suivantes :

// SFR : là sans connaissance préalable c'est difficile de comprendre
//       Soit modifier l'intro en disant que le vocabulaire R1CS, QAP, etc doit être connu, 
//       Soit donner plus d'explications par ici.

1- Transformer le polynôme en un circuit arithmétique
2- Créer un modèle R1CS à partir du circuit arithmétique
3- Générer le modèle QAP à partir du modèle R1CS
4- Demander au trusted setup de générer les clés de vérification
5- Générer la preuve de connaissance (Prouver)
6- Verifier la preuve de connaissance (Vérifier)

À savoir que le protocole Groth16 concerne la partie 4,5 et 6. Néanmoins, nous voulions automatiser la modélisation de notre problème sous forme QAP en donnant uniquement un polynôme au début du programme.
Voici en une image le workflow de l'implantation :

![workflow](src/intro.png)

Sur le schéma ci-dessus la partie 1, 2 et 3 sont faites dans le rectangle blanc, c'est ici que l'on définit un polynôme et ensuite le programme détermine les données nécessaires pour le trusted setup, le prover et le vérifier.

## Apparter sur les bibliothèques utilisées:

Dans ce programme, nous utilisons les bibliothèques numpy et galois pour faire des calculs dans des champs finits et py_ecc pour travailler sur les courbes elliptiques. De plus nous avons programmé nos propres fonctions dans un fichier zktool.py qui est disponible sur GitHub et à la fin de cet article. 
// SFR : lister ici les fonctions dont tu as besoin

In [1]:
import zktool as zk
import numpy as np
import galois
from py_ecc.bn128 import curve_order
import galois
import numpy as np

## Initialisation du corps de Galois

Nous devons choisir un corps de Galois fini pour effectuer nos calculs. Nous allons utiliser le corps de Galois fini de la courbe BN128. Pour cela nous allons utiliser la bibliothèque `galois`. Il est important d'utiliser le même corps de Galois fini pour le prouveur et le vérifieur. Ici p=curve_order est l'ordre de la courbe BN128.

A noter que le temps d'exécution de la prochaine cellule est long.

In [2]:
p=curve_order # p=21888242871839275222246405745257275088548364400416034343698204186575808495617
GF = galois.GF(p) # Long

## Définition du polynome qu'on souhaite prouver

Le format de l'équation est le suivant : a_nx^x+a_n-1x^n-1+…+a_1x+a_0
Exemple : 2x²+3x+1 Actuellement les coefficients sont des entiers positifs.
Voici le fonctionnement de notre fonction `decompose_polynomial()`:

![workflow](src/1.png)

In [3]:
print("=================== GROTH16 IMPLEMENTATION ===================")

eq_input = input("Enter the equation (ex: x^3 + x + 5):") 
print("Equation: ", eq_input, "\n")

equations, solution_array = zk.decompose_polynomial(eq_input)

print("Arithmetic circuit modeling:")
for equation in equations:
    print(equation)

print("\nSet of variables =", solution_array)

=================== GROTH16 IMPLEMENTATION ===================


## R1CS

Pour continuer nous devons utliser le model R1CS. Pour cela nous allons utiliser la bibliothèque `zktool` et la fonction `generate_r1cs` pour générer le model R1CS à partir du circuit arithmétique. Cette fonction prend en paramètre le circuit arithmétique et retourne le model R1CS.
![workflow](src/2.png)

In [4]:
L, R, O = zk.generate_r1cs(equations, solution_array)

print("L=")
for line in L:
    print(line)
print(" ")

print("R=")
for line in R:
    print(line)
print(" ")

print("O=")
for line in O:
    print(line)
print(" ")

L=
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 1, 0]
[5, 0, 0, 0, 0, 1]
 
R=
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
 
O=
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 0, 0]
 


## R1CS to QAP

Une fois les matrices du modèle R1CS générées, nous allons les transformer en matrices suivant le modèle QAP. Pour cela, nous allons déterminer les polynômes avec l'interpolation de Lagrange.

Voici un exemple pour transformer la matrice L_galois, qui vaut aussi L, car chaque coefficient dans la matrice est déjà compris entre [0,1-p] en U, qui est une matrice de polynômes. Ici, nous avons tronqué les coefficients des polynômes pour avoir un affichage plus clair. 
![workflow](src/3.png)

In [5]:
num_eq = len(L)

# Convert the matrices to be convertible in GF (each value is brought between [0, p-1])
# Convert the matrices into Galois matrices
L_galois = GF(np.array(L) % p)
R_galois = GF(np.array(R) % p)
O_galois = GF(np.array(O) % p)

U = []
V = []
W = []
for i in range(len(solution_array)):
    U.append(zk.interpolate_lagrange(L_galois[:, i], GF, num_eq))
    V.append(zk.interpolate_lagrange(R_galois[:, i], GF, num_eq))
    W.append(zk.interpolate_lagrange(O_galois[:, i], GF, num_eq))

print("Matrix U:")
for line in U:
    print(line)

print("\nMatrix V:")
for line in V:
    print(line)

print("\nMatrix W:")
for line in W:
    print(line)

Matrix U:
[3648040478639879203707734290876212514758060733402672390616367364429301415937
 21888242871839275222246405745257275088548364400416034343698204186575808495612
 18240202393199396018538671454381062573790303667013361953081836822146507079690
 21888242871839275222246405745257275088548364400416034343698204186575808495612]
[0 0 0 0]
[14592161914559516814830937163504850059032242933610689562465469457717205663744
 5
 7296080957279758407415468581752425029516121466805344781232734728858602831861
 8]
[10944121435919637611123202872628637544274182200208017171849102093287904247809
 21888242871839275222246405745257275088548364400416034343698204186575808495613
 10944121435919637611123202872628637544274182200208017171849102093287904247818
 21888242871839275222246405745257275088548364400416034343698204186575808495611]
[10944121435919637611123202872628637544274182200208017171849102093287904247808
 10944121435919637611123202872628637544274182200208017171849102093287904247812
 218882428718392752222464

## Évalutation en un point

On choisit un nombre au hasard dans le corps de Galois fini et on déroule l'ensemble de nos équations pour calculer le vecteur solution. Ici, on prend comme point de départ x=5.

zk.compute_solution_vector permet d'évaluer le solution_array avec comme point de départ x=5.

![workflow](src/4.png)

In [6]:
values = {"x": 5}
a = zk.compute_solution_vector(solution_array, equations, values)
a = np.array(a) % p

print("\nSolution Vector:", a)
print("Vector with variable", solution_array)


Solution Vector: [1 135 5 25 125 130]
Vector with variable ['1', 'out', 'x', 'v1', 'v2', 'v3']


## Application de groth16

Maintenant que nous avons toutes les données nécessaires nous pouvons appliquer groth16, c'est à dire notre vecteur solution évalué en un point et notre model QAP. On peut transmettre nos matrices à la fonction `generate_trusted_setup` de la bibliothèque `zktool` pour générer les clés de vérification. Cette fonction prend en paramètre les matrices du model QAP et retourne les clés de vérification.

![workflow](src/5.png)

Ensuite nous pouvons utiliser la fonction `generate_proof` de la bibliothèque `zktool` pour générer la preuve de connaissance. Cette fonction prend en paramètre les clés de vérification, le vecteur solution évalué en un point et le model QAP. Elle retourne la preuve de connaissance.

![workflow](src/6.png)

Et enfin, nous pouvons utiliser la fonction `verify_proof` de la bibliothèque `zktool` pour vérifier la preuve de connaissance. Cette fonction prend en paramètre les clés de vérification, le vecteur solution évalué en un point, le model QAP et la preuve de connaissance. Elle retourne un booléen indiquant si la preuve de connaissance est valide ou non.

![workflow](src/7.png)

In [7]:
zk.init(GF)

U = np.array(U)
V = np.array(V)
W = np.array(W)

a = GF(a)
l = 1 # Publicly provide the first 2 elements of a, the rest is private
sigma_1, sigma_2 = zk.trusted_setup(U, V, W, l)
pi = zk.prover(U, V, W, l, sigma_1, sigma_2, a)
isTrue = zk.verifier(pi, a, sigma_2, sigma_1, l)
print(isTrue)

x =  75
alpha =  2
beta =  3
gamma =  5
delta =  11

============= Prover =============
A_1= (5007602948002553469188036854420775706275968160992803697339877036351367343682, 18416552614392113266381640926605678220410225438237484861022484597644638788899)
B_2= ((13235053251753173958467675402624206475371210159860213279280275675358774651412, 19636830560278721114685701923125581805643832425963538909810871648303095584480), (2844054701750252050655405984433178353653775737812896814285685839499536073112, 6271240352858791095865723664896546107464021415293552513864711282796755706530))
C_1= (15651873788416093060218677531124579982017950547248813017969348280099555404525, 21692469522036355297773783805225131010452868703025703788025072291105621548540)

============= Verifier =============
True
